# Shobhit Bhatnagar, Roll No. 14UEC096

## Simplex Algorithm Implementation

In [1]:
import numpy as np

def simplex(of,basis,tableau):
    # Get the number of rows and columns in the tableau:
    n_rows = tableau.shape[0]
    n_cols = tableau.shape[1]
    # Start the simplex algorithm:
    # Compute zj-cj. If zj - cj >= 0 for all columns then current 
    # solution is optimal solution.
    check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) * 
        tableau[:,0:n_cols-1],axis=0) - of
    count = 0
    while ~np.all(check >= 0):
        # Determine the pivot column:
        # The pivot column is the column corresponding to 
        # minimum zj-cj.
        pivot_col = np.argmin(check)
        # Determine the positive elements in the pivot column.
        # If there are no positive elements in the pivot column  
        # then the optimal solution is unbounded.
        positive_rows = np.where(tableau[:,pivot_col] > 0)[0]
        if positive_rows.size == 0:
            print('Unbounded Solution!')
            break
        # Determine the pivot row:
        divide=(tableau[positive_rows,n_cols-1]
            /tableau[positive_rows,pivot_col])
        pivot_row = positive_rows[np.where(divide 
            == divide.min())[0][-1]]
        # Update the basis:
        basis[pivot_row] = pivot_col
        # Perform gaussian elimination to make pivot element one and
        # elements above and below it zero:
        tableau[pivot_row,:]=(tableau[pivot_row,:]
            /tableau[pivot_row,pivot_col])
        for row in range(n_rows):
            if row != pivot_row:
                tableau[row,:] = (tableau[row,:] 
                    - tableau[row,pivot_col]*tableau[pivot_row,:])
        # Determine zj-cj
        check = np.sum(np.reshape(of[list(basis)],(n_rows,1)) * 
            tableau[:,0:n_cols-1],axis=0) - of
        count += 1
        print('Step %d' % count)
        print(tableau)
    return basis,tableau

def get_solution(of,basis,tableau):
    # Get the no of columns in the tableau:
    n_cols = tableau.shape[1]
    # Get the optimal solution:
    solution = np.zeros(of.size)
    solution[list(basis)] = tableau[:,n_cols-1]
    # Determine the optimal value:
    value = np.sum(of[list(basis)] * tableau[:,n_cols-1])
    return solution,value


## Question 1

Solve the following L.P.P. by Simplex Method.

Maximize $z=4x_1+5x_2+9x_3+11x_4$ subject to
$$
\begin{aligned}
x_1+x_2+x_3+x_4 &\leq 15,  \\
7x_1 + 5x_2+3x_3+2x_4 &\leq 120, \\
3x_1+5x_2+10x_3+15x_4 &\leq 100, \\
x_1,x_2,x_3,x_4 &\geq 0
\end{aligned}
$$

### Solution

We can convert the problem into the standard form by adding slack variables $x_5,x_6$ and $x_7$. 

Maximize $z = 4x_1+5x_2+9x_3+11x_4+0x_5+0x_6+0x_7$ subject to
$$\begin{aligned}
x_1+x_2+x_3+x_4+x_5 &= 15, \\
7x_1+5x_2+3x_3+2x_4 +x_6 &= 120, \\
3x_1+5x_2+10x_3+15x_4+x_7 &= 100, \\
x_1,x_2,x_3,x_4,x_5,x_6,x_7 &\geq 0
\end{aligned}$$

We are now ready to solve the problem.

In [2]:
def question1():
    # Define the tableau:
    tableau = np.array([
        [1.0,1,1,1,1,0,0,15],
        [7,5,3,2,0,1,0,120],
        [3,5,10,15,0,0,1,100]
    ])
    # Define the objective function and the initial basis:
    of = np.array([4,5,9,11,0,0,0])
    basis = np.array([4,5,6])
    # Run the simplex algorithm:
    basis,tableau = simplex(of,basis,tableau)
    # Get the optimal soultion:
    optimal_solution,optimal_value = get_solution(of,basis,tableau)
    # Print the final tableau:
    print('The final basis is:')
    print(basis)
    # Print the results:
    print('Solution\nx1=%0.2f, x2=%0.2f, x3=%0.2f, x4=%0.2f, z=%0.4f' 
        % (optimal_solution[0],optimal_solution[1],optimal_solution[2],
            optimal_solution[3],optimal_value))
question1()

Step 1
[[  8.00000000e-01   6.66666667e-01   3.33333333e-01   0.00000000e+00
    1.00000000e+00   0.00000000e+00  -6.66666667e-02   8.33333333e+00]
 [  6.60000000e+00   4.33333333e+00   1.66666667e+00   0.00000000e+00
    0.00000000e+00   1.00000000e+00  -1.33333333e-01   1.06666667e+02]
 [  2.00000000e-01   3.33333333e-01   6.66666667e-01   1.00000000e+00
    0.00000000e+00   0.00000000e+00   6.66666667e-02   6.66666667e+00]]
Step 2
[[  1.           0.83333333   0.41666667   0.           1.25         0.
   -0.08333333  10.41666667]
 [  0.          -1.16666667  -1.08333333   0.          -8.25         1.
    0.41666667  37.91666667]
 [  0.           0.16666667   0.58333333   1.          -0.25         0.
    0.08333333   4.58333333]]
Step 3
[[  1.           0.71428571   0.          -0.71428571   1.42857143   0.
   -0.14285714   7.14285714]
 [  0.          -0.85714286   0.           1.85714286  -8.71428571   1.
    0.57142857  46.42857143]
 [  0.           0.28571429   1.           1.7142

The optimal solution is $x_1=7.1428,x_2=0,x_3=7.8571,x_4=0$ and the optimum value is $z=99.2857$.

## Question 2

Solve the following L.P.P. by Big M method.

Maximize $z = 2x_1+4x_2+x_3$ subject to
$$\begin{aligned}
x_1 + 2x_2 -x_3 &\leq 5, \\
2x_1 -x_2 +2x_3 &= 2, \\
-x_1 +2x_2+2x_3 &\geq 1, \\
x_1,x_2,x_3 &\geq 0
\end{aligned}$$

### Solution

To convert the problem into the standard form we introduce a slack variable $x_4$, a surplus variable $x_6$ and two artificial variables $x_5$ and $x_7$. 

Maximize $z = 2x_1+4x_2+x_3 - M x_5-Mx_7$ subject to
$$\begin{aligned}
x_1+2x_2-x_3+x_4 &= 5, \\
2x_2-x_2+2x_3 +x_5 &= 2, \\
-x_1+2x_2+2x_3-x_6+x_7 &= 1, \\
x_1,x_2,x_3,x_4,x_5,x_6,x_7 &\geq 0
\end{aligned}$$

We can now proceed to solve the problem as before.

In [3]:
def question2():
    # Big M Method
    # Define the tableau:
    tableau = np.array([
        [1.0,2,-1,1,0,0,0,5],
        [2,-1,2,0,1,0,0,2],
        [-1,2,2,0,0,-1,1,1]
    ])
    # Define the objective function: We will associate a very large
    # penalty with the artificial variables:
    of = np.array([2,4,1,0,-10000,0,-10000])
    # Define the initial basis:
    basis = np.array([3,4,6])
    # Run the simplex algorithm:
    basis,tableau = simplex(of,basis,tableau)
    # Get the optimal solution:
    optimal_solution,optimal_value = get_solution(of,basis,tableau)
    # Print the final tableau:
    print('The final basis is:')
    print(basis)
    # Print the results:
    print('Solution\nx1=%0.2f,\nx2=%0.2f,\nx3=%0.2f,\nz=%0.4f' 
        % (optimal_solution[0],optimal_solution[1],
            optimal_solution[2],optimal_value))
question2()

Step 1
[[ 0.5  3.   0.   1.   0.  -0.5  0.5  5.5]
 [ 3.  -3.   0.   0.   1.   1.  -1.   1. ]
 [-0.5  1.   1.   0.   0.  -0.5  0.5  0.5]]
Step 2
[[ 0.          3.5         0.          1.         -0.16666667 -0.66666667
   0.66666667  5.33333333]
 [ 1.         -1.          0.          0.          0.33333333  0.33333333
  -0.33333333  0.33333333]
 [ 0.          0.5         1.          0.          0.16666667 -0.33333333
   0.33333333  0.66666667]]
Step 3
[[ 0.          0.         -7.          1.         -1.33333333  1.66666667
  -1.66666667  0.66666667]
 [ 1.          0.          2.          0.          0.66666667 -0.33333333
   0.33333333  1.66666667]
 [ 0.          1.          2.          0.          0.33333333 -0.66666667
   0.66666667  1.33333333]]
Step 4
[[ 0.   0.  -4.2  0.6 -0.8  1.  -1.   0.4]
 [ 1.   0.   0.6  0.2  0.4  0.   0.   1.8]
 [ 0.   1.  -0.8  0.4 -0.2  0.   0.   1.6]]
Step 5
[[  7.           0.           0.           2.           2.           1.
   -1.          13.      

The optimal solution is $x_1=0,x_2=4,x_3=3$ and the optimal value is $z=19$.

## Question 3

Solve the following L.P.P. by Two phase method.

Maximize $z=2x_1+x_2+x_3$ subject to
$$\begin{aligned}
4x_1+6x_2+3x_3 &\leq 8, \\
3x_1-6x_2-4x_3 &\leq 1, \\
2x_1+3x_2-5x_3 &\geq 4, \\
x_1,x_2,x_3 &\geq 0
\end{aligned}$$

### Solution

In phase I, we will solve the following problem

Maximize $z = -x_7$ subject to

$$
\begin{aligned}
4x_1+6x_2+3x_3+x_4 &= 8, \\
3x_1-6x_2-4x_3+x_5 &= 1, \\
2x_1+3x_2-5x_3-x_6+x_7 &= 4, \\
x_1,x_2,x_3,x_4,x_5,x_6,x_7 &\geq 0
\end{aligned}
$$

where $x_4,x_5$ are slack variables, $x_6$ is a surplus variable and $x_7$ is an artificial variable.

In [4]:
def question3():
    # 2 Phase Method
    # Define the tableau:
    tableau = np.array([
        [4.0,6,3,1,0,0,0,8],
        [3,-6,-4,0,1,0,0,1],
        [2,3,-5,0,0,-1,1,4]
    ])
    # Phase I:
    # Define the initial basis and the objective function:
    basis = np.array([3,4,6])
    of = np.array([0,0,0,0,0,0,-1])
    # Run the simplex algorithm:
    print('Phase I:')
    basis, tableau = simplex(of,basis,tableau)
    # Phase II:
    # Define new objective function:
    of = np.array([2,1,1,0,0,0])
    # Remove the artificial column:
    tableau = np.c_[tableau[:,0:6],tableau[:,7]]
    # Run simplex algorithm again:
    print('Phase II:')
    basis, tableau = simplex(of,basis,tableau)
    # Get the optimal solution:
    optimal_solution,optimal_value = get_solution(of,basis,tableau)
    # Print the final tableau:
    print('The final basis is:')
    print(basis)
    # Print the results:
    print('Solution\nx1=%0.2f,\nx2=%0.2f,\nx3=%0.2f,\nz=%0.4f' 
        % (optimal_solution[0],optimal_solution[1],
            optimal_solution[2],optimal_value))
question3()

Phase I:
Step 1
[[  0.           0.          13.           1.           0.           2.
   -2.           0.        ]
 [  7.           0.         -14.           0.           1.          -2.
    2.           9.        ]
 [  0.66666667   1.          -1.66666667   0.           0.          -0.33333333
    0.33333333   1.33333333]]
Phase II:
Step 1
[[ 0.          0.          1.          0.07692308  0.          0.15384615
   0.        ]
 [ 7.          0.          0.          1.07692308  1.          0.15384615
   9.        ]
 [ 0.66666667  1.          0.          0.12820513  0.         -0.07692308
   1.33333333]]
Step 2
[[ 0.          0.          1.          0.07692308  0.          0.15384615
   0.        ]
 [ 1.          0.          0.          0.15384615  0.14285714  0.02197802
   1.28571429]
 [ 0.          1.          0.          0.02564103 -0.0952381  -0.09157509
   0.47619048]]
The final basis is:
[2 0 1]
Solution
x1=1.29,
x2=0.48,
x3=0.00,
z=3.0476


The optimal solution is $x_1=1.29, x_2=0.48,x_3=0$ and the optimal value is $z=3.0476$.

## Question 4

By solving the dual of the following L.P.P., find the minimum value of $z$.

Minimize $z = 3x_1-2x_2+4x_3$ subject to
$$\begin{aligned}
3x_1+5x_2+4x_3 &\geq 7, \\
6x_1+x_2+x_3 &\geq 4, \\
7x_1-2x_2-x_3 &\leq 10, \\
4x_1+7x_2-2x_3 &\geq 2, \\
x_1-2x_2 + 5x_3 &\geq 4, \\
x_1,x_2,x_3 &\geq 0
\end{aligned}$$

### Solution

The dual of the given problem is

Maximize $z=7y_1+4y_2-10y_3+2y_4+3y_5$ subject to
$$\begin{aligned}
3y_1+6y_2-7y_3+4y_4+y_5 &\leq 3, \\
-5y_1-y_2-2y_3-7y_4+2y_5 &\geq 2, \\
4y_1+y_2+y_3-2y_4+5y_5 &\leq 4, \\
y_1,y_2,y_3,y_5,y_5 &\geq 0
\end{aligned}$$

We will use Big-M method to solve this problem. In the standard form,

Maximize $z=7y_1+4y_2-10y_3+2y_4+3y_5+0y_6+0y_7-My_8+0y_9$ subject to

$$\begin{aligned}
3y_1+6y_2-7y_3+4y_4+y_5 +y_6&= 3, \\
-5y_1-y_2-2y_3-7y_4+2y_5 -y_7+y_8&= 2, \\
4y_1+y_2+y_3-2y_4+5y_5 +y_9&= 4, \\
y_1,y_2,y_3,y_4,y_5,y_6,y_7,y_8,y_9 &\geq 0
\end{aligned}$$

Here $y_6,y_9$ are slack variables, $y_7$ is a surplus variable and $y_8$ is an artificial variable.

In [5]:
def question4():
    # Minimum value by solving the dual of the given problem:
    # The tableau of the dual problem is:
    tableau = np.array([
        [3.0,6,-7,4,1,1,0,0,0,3],
        [-5,-1,-2,-7,2,0,-1,1,0,2],
        [4,1,1,-2,5,0,0,0,1,4]
    ])
    # The objective function of the dual problem is:
    of = np.array([7,4,-10,2,3,0,0,-10000,0])
    basis = np.array([5,7,8])
    # Solve the problem:
    basis, tableau = simplex(of,basis,tableau)
    optimal_solution, optimal_value = get_solution(of,basis,tableau)
    # Check if the artificial variable is still remaining in basis:
    print('The final basis is:')
    print(basis)
    if np.any(basis == 7):
        print('The dual problem has no feasible solution.')
    else:
        print('z=%0.2f' % optimal_value)

question4()

Step 1
[[ 2.2  5.8 -7.2  4.4  0.   1.   0.   0.  -0.2  2.2]
 [-6.6 -1.4 -2.4 -6.2  0.   0.  -1.   1.  -0.4  0.4]
 [ 0.8  0.2  0.2 -0.4  1.   0.   0.   0.   0.2  0.8]]
The final basis is:
[5 7 4]
The dual problem has no feasible solution.


Since the dual problem has no feasible solution, the original problem has unbounded optimal solution.